In [3]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:90%;}
div.CodeMirror {font-family:Consolas; font-size:18pt;}
div.output {font-size:19pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:19pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
span.toc-item-num{display:none;}
div.text_cell_render ul li{font-size:16pt;padding:5px;}
div.CodeMirror-lines > div {padding-left:10px;}
table.dataframe{font-size:13px;}
</style>
"""))

<b><font color="red" size="6">ch10. 데이터 베이스 연동</font></b>

# 1절. SQLite 데이터 베이스 연결
- SQLite 데이터 베이스는 별도의 DBMS없이 SQL을 이용하여 DB 엑세스 할 수 있도록 만든 간단한 디스크 기반의 DB 제공
- C 라이브러리
- SQLite는 프로토타입을 만들 때 사용
- 프로젝트 단계 : 분석 → 설계 → 구현 → 테스트 → 고객에게 배포 → 유지보수
-               프로토타입(SQLite)  시제품     완제품(Oracle, MySql, Maria, ...)
-
- [DB browser for SQLite](https://sqlitebrowser.org/dl/)
## 1.1 SQLite broswer 설치 및 sqlite3 패키지 load

In [5]:
import sqlite3
sqlite3.sqlite_version # sqlite3 패키지의 버전

'3.40.1'

In [6]:
import pandas as pd
pd.__version__

'1.5.3'

In [7]:
import numpy as np
np.__version__

'1.23.5'

## 1.2 데이터베이스 연결
- SQLite로 DB 연결시, DB파일이 있으면 연결, DB파일이 없으면 빈 DB 파일 생성

In [8]:
# DB 연결
conn = sqlite3.connect('data/ch10_example.db')
conn

In [9]:
# 커서 객체 생성. 커서는 SQL문 실행시키고, 결과데이터를 조회(select, 그외)하는데 사용하는 객체
cursor = conn.cursor()
cursor

In [10]:
cursor.execute('''
    CREATE TABLE MEMBER (
        NAME TEXT,
        AGE  INT,
        EMAIL TEXT
    )
''')

In [11]:
cursor.execute('DROP TABLE MEMBER')

In [12]:
cursor.execute('''
    CREATE TABLE MEMBER (
        NAME TEXT,
        AGE  INT,
        EMAIL TEXT
    )
''')

In [13]:
cursor.execute('INSERT INTO MEMBER VALUES (\'홍길동\', 20, \'h@h.com\')')
print('수행 결과 행수 :', cursor.rowcount)
sql = "INSERT INTO MEMBER VALUES ('신길동', 25, 'S@S.COM')"
cursor.execute(sql)
print('수행 결과 행수 :', cursor.rowcount)
cursor.execute('INSERT INTO MEMBER VALUES (\'신림동\', 30, \'l@h.com\')')
print('수행 결과 행수 :', cursor.rowcount)

수행 결과 행수 : 1
수행 결과 행수 : 1
수행 결과 행수 : 1


In [14]:
conn.commit() # 反. conn.rollback() DML문에서만 commit

In [15]:
# SQL전송 결과는 cursor가 가리킴
cursor.execute("SELECT * FROM MEMBER ORDER BY NAME")

In [16]:
# insert, update, delete 문 실행결과 : cursor.rowcount
# select문 실행 결과를 받는 함수들
    ## fetchone() : 결과를 한행씩 받을 때 (튜플)
    ## fetchall() : 결과를 모두 받을 때 (튜플 list)
    ## fetchmany(n) : 결과를 n행 받을 때 (튜플 list)
print(cursor.fetchall())

[('신길동', 25, 'S@S.COM'), ('신림동', 30, 'l@h.com'), ('홍길동', 20, 'h@h.com')]


In [17]:
print(cursor.fetchall()) # 한번 소요된 cursor 객체는 다시 fetch할 수 없음

[]


In [18]:
cursor.execute("SELECT NAME, AGE, EMAIL FROM MEMBER ORDER BY AGE")
members = cursor.fetchall()
members # 튜플 리스트

[('홍길동', 20, 'h@h.com'), ('신길동', 25, 'S@S.COM'), ('신림동', 30, 'l@h.com')]

In [20]:
for member in members:
    print(member)

('홍길동', 20, 'h@h.com')
('신길동', 25, 'S@S.COM')
('신림동', 30, 'l@h.com')


In [22]:
# 한줄씩 읽기
cursor.execute("SELECT NAME, AGE, EMAIL FROM MEMBER ORDER BY AGE")
member_list = []
while True:
    member = cursor.fetchone() # SQL문 수행 결과 한줄 가져오기
    if member is None:
        break
    #print(member)
    member_list.append({'name':member[0], 'age': member[1], 'email': member[2]})

In [23]:
member_list

[{'name': '홍길동', 'age': 20, 'email': 'h@h.com'},
 {'name': '신길동', 'age': 25, 'email': 'S@S.COM'},
 {'name': '신림동', 'age': 30, 'email': 'l@h.com'}]

In [25]:
# 최상위 n행 읽기
cursor.execute("SELECT NAME, AGE, EMAIL FROM MEMBER ORDER BY AGE")
for member in cursor.fetchmany(10):
    print(member)

('홍길동', 20, 'h@h.com')
('신길동', 25, 'S@S.COM')
('신림동', 30, 'l@h.com')


In [26]:
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self, name, age, email):
        self.name = name
        self.age  = age
        self.email= email
    def as_dic(self): # insert문 전송시 필요
        return {'name':self.name,
                'age':self.age,
                'email':self.email}
    def __str__(self):
        return "{}\t{}\t{}".format(self.name, self.age, self.email)
def to_member(*row): # 튜플 데이터를 매개변수로 받아 Member형 객체로 return
    return Member(row[0], row[1], row[2])

In [27]:
dbreadmember = ('홍길동', 20, 'h@h.com')
m = to_member(*dbreadmember) # 튜플 언패킹
print(m)
print(m.as_dic())

홍길동	20	h@h.com
{'name': '홍길동', 'age': 20, 'email': 'h@h.com'}
